In [49]:
import pandas as pd
import os

In [50]:
file_list = []
for dirname, _, filenames in os.walk(f"{AMZN_folder}"):
    for filename in filenames:
        file_list.append(os.path.join(dirname,filename))

In [51]:
list_num = 4
print(file_list[list_num])
pd.read_csv(file_list[list_num])

../data/timeseries/AMZN/1006000 - Statement - Consolidated Balance Sheets.csv


,consolidatedbalancesheets,2020-03-31,2019-09-30,2019-06-30,2019-03-31,2018-09-30,2018-06-30,2018-03-31,2017-09-30,2017-06-30,...,2013-03-31,2012-09-30,2012-06-30,2012-03-31,2011-09-30,2011-06-30,2011-03-31,2010-09-30,2010-06-30,2010-03-31
0,us-gaap_cashandcashequivalentsatcarryingvalue,2.720100e+10,2.325500e+10,2.261600e+10,2.311500e+10,2.042500e+10,1.982300e+10,1.667600e+10,1.276700e+10,1.320300e+10,...,4.481000e+09,2.980000e+09,2.335000e+09,2.288000e+09,2.823000e+09,2.047000e+09,2.641000e+09,1.539000e+09,1.629000e+09,1.844000e+09
1,us-gaap_marketablesecuritiescurrent,2.209100e+10,2.014600e+10,1.884700e+10,1.390500e+10,9.340000e+09,7.227000e+09,8.287000e+09,1.154300e+10,8.248000e+09,...,3.414000e+09,2.268000e+09,2.635000e+09,3.427000e+09,3.503000e+09,4.308000e+09,4.240000e+09,4.346000e+09,3.479000e+09,3.219000e+09
2,us-gaap_inventorynet,1.885700e+10,1.876600e+10,1.858000e+10,1.643200e+10,1.586200e+10,1.482400e+10,1.384000e+10,1.371100e+10,1.151000e+10,...,5.395000e+09,5.065000e+09,4.380000e+09,4.255000e+09,3.770000e+09,3.229000e+09,2.888000e+09,2.515000e+09,1.940000e+09,1.820000e+09
3,us-gaap_accountsreceivablenetcurrent,1.783600e+10,1.688700e+10,1.674700e+10,1.597900e+10,1.425800e+10,1.260700e+10,1.202600e+10,1.055700e+10,8.046000e+09,...,2.516000e+09,2.392000e+09,2.035000e+09,1.813000e+09,1.496000e+09,1.438000e+09,1.304000e+09,9.590000e+08,8.050000e+08,8.150000e+08
4,us-gaap_assetscurrent,8.598500e+10,7.905400e+10,7.679000e+10,6.943100e+10,5.988500e+10,5.448100e+10,5.082900e+10,4.857800e+10,4.100700e+10,...,1.631300e+10,1.311800e+10,1.179300e+10,1.215400e+10,1.190400e+10,1.127900e+10,1.128800e+10,9.559000e+09,8.118000e+09,7.964000e+09
5,us-gaap_propertyplantandequipmentnet,7.777900e+10,6.766200e+10,6.472300e+10,6.104800e+10,5.801900e+10,5.476800e+10,5.233100e+10,4.533500e+10,3.708300e+10,...,7.674000e+09,5.662000e+09,5.097000e+09,4.653000e+09,3.999000e+09,3.470000e+09,2.902000e+09,2.099000e+09,1.704000e+09,1.436000e+09
6,us-gaap_operatingleaserightofuseasset,2.627900e+10,2.311400e+10,2.164900e+10,2.013200e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,us-gaap_goodwill,1.473900e+10,1.473400e+10,1.472700e+10,1.470800e+10,1.455300e+10,1.394400e+10,1.338800e+10,1.327100e+10,4.254000e+09,...,2.535000e+09,2.540000e+09,2.521000e+09,1.970000e+09,1.934000e+09,1.909000e+09,1.513000e+09,1.277000e+09,1.229000e+09,1.234000e+09
8,us-gaap_otherassetsnoncurrent,1.645600e+10,1.453500e+10,1.346200e+10,1.278300e+10,1.123800e+10,1.090700e+10,9.814000e+09,8.083000e+09,5.437000e+09,...,1.732000e+09,1.476000e+09,1.585000e+09,1.535000e+09,1.190000e+09,1.253000e+09,1.151000e+09,1.184000e+09,1.317000e+09,1.392000e+09
9,us-gaap_assets,2.212380e+11,1.990990e+11,1.913510e+11,1.781020e+11,1.436950e+11,1.341000e+11,1.263620e+11,1.152670e+11,8.778100e+10,...,2.837700e+10,2.283400e+10,2.102200e+10,2.033900e+10,1.905400e+10,1.794100e+10,1.688200e+10,1.416200e+10,1.239700e+10,1.204200e+10


In [40]:
path = '../⁨data⁩/csv⁩/AMZN⁩/2020-03-31 (10-Q)⁩'
file = '1006000 - Statement - Consolidated Balance Sheets.csv'
pd.read_csv(os.path.join(path,file))

FileNotFoundError: [Errno 2] File ../⁨data⁩/csv⁩/AMZN⁩/2020-03-31 (10-Q)⁩/1006000 - Statement - Consolidated Balance Sheets.csv does not exist: '../\u2068data\u2069/csv\u2069/AMZN\u2069/2020-03-31 (10-Q)\u2069/1006000 - Statement - Consolidated Balance Sheets.csv'